In [29]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pickle

import warnings
warnings.filterwarnings('ignore')

In [30]:
def onehot(df: pd.DataFrame):
    encoder = OneHotEncoder(drop='first').fit(pd.DataFrame(df))
    encoded = encoder.transform(pd.DataFrame(df)).toarray()
    cols = encoder.get_feature_names_out(input_features=df.columns)
    cols = list(map(normit, cols))  
    onehot_encoded_cats = pd.DataFrame(encoded, columns=cols)
    return onehot_encoded_cats


def normit(x: str):
    return x.replace(' ', '_').replace('/', '_').lower()


def get_height(x: str):
    feet, inch = x.split("'")
    inch = inch.rstrip('"')
    return int(feet)*30.48 + int(inch)*2.54


def get_value(x):
    x = str(x)
    if x[-1] == 'M':
        x = x[1:-1]
        return 1000000 * float(x)
    elif x[-1] == 'K':
        x = x[1:-1]
        return 1000 * float(x)
    else:
        x = x[1:]
        return float(x)


def get_stars(x):
    return int(x[0])

# commented lines are from initial model
def clean_fifa_df(df: pd.DataFrame):
    df.columns = list(map(normit, df.columns))
    # dropped = df[[
    #     'age', 'growth', 'value', 'wage', 'reactions', 'power', 'shot_power', 'vision', 
    #     'composure', 'total_stats', 'base_stats', 'pas', 'dri', 'phy', 
    #     'ova']]
    dropped = df[['age', 'bp', 'height', 'weight', 'foot', 'growth', 'value', 'wage',
       'release_clause', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'ova']]

    dropped['composure'] = dropped['composure'].fillna(np.mean(dropped['composure'])).copy()
    dropped['a_w'] = dropped['a_w'].fillna('Medium').copy()
    dropped['d_w'] = dropped['d_w'].fillna('Medium').copy()
    dropped = dropped.dropna(subset=['vision'])
    dropped['height'] = dropped['height'].apply(get_height)
    dropped['weight_lbs'] = dropped['weight'].str.rstrip("lbs").astype(int)
    dropped = dropped.drop(['weight'], axis=1)
    dropped['value'] = dropped['value'].apply(get_value)
    dropped['wage'] = dropped['wage'].apply(get_value)
    dropped['release_clause'] = dropped['release_clause'].apply(get_value) 
    dropped['w_f'] = dropped['w_f'].apply(get_stars)
    dropped['sm'] = dropped['sm'].apply(get_stars)
    dropped['ir'] = dropped['ir'].apply(get_stars)

    return dropped


In [31]:
def preprocess(df: pd.DataFrame):
    cleaned = clean_fifa_df(df)
    y = cleaned['ova']
    cleaned = cleaned.drop(['ova'], axis=1)
    nums = cleaned.select_dtypes(include=np.number)
    cats = cleaned.select_dtypes(include=object)
    # nums = minmax(nums)
    # cats = onehot(cats)
    X = pd.concat((nums, cats), axis=1)
    return X, y

In [32]:
# minmax on x_train and same transformer on test, NOT minmax an train+test   ?????????????

In [33]:
validate = pd.read_csv('fifa21_validate.csv')
train = pd.read_csv('fifa21_train.csv')
validate.columns == train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [34]:
def scale_and_onehot(df, scaler: MinMaxScaler):
    numsi = df.select_dtypes(include=np.number)
    cats = df.select_dtypes(include=object)
    cats = onehot(cats)
    nums = scaler.transform(numsi)
    nums = pd.DataFrame(nums, columns=numsi.columns)
    return pd.concat((nums, cats), axis=1)
    
X, y = preprocess(train)
X2, y2 = preprocess(validate)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

nums_train = X_train.select_dtypes(include=np.number)

minmax = MinMaxScaler().fit(nums_train)

X_train = scale_and_onehot(X_train, minmax)
X_test = scale_and_onehot(X_test, minmax)
X2 = scale_and_onehot(X2, minmax)

In [36]:
lm = linear_model.LinearRegression()
lm.fit(X_train, y_train)

preds_test = lm.predict(X_test)
print(r2_score(y_test, preds_test))

preds = lm.predict(X2)
print(r2_score(y2, preds))

0.918610521630802
0.9163031339732207


In [37]:
# trying to figure out where the validating goes wrong

pd.set_option('display.max_columns', None)

print(y.shape)
print(y2.shape)
display(X.describe())
print(X.shape)
print(X_train.shape)
print(X_test.shape)
display(X2.describe())
print(X2.shape)

(11660,)
(1996,)


,age,height,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w_f,sm,ir,pac,sho,pas,dri,def,phy,weight_lbs
count,11660.000000,11660.000000,11660.000000,1.166000e+04,11660.000000,1.166000e+04,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000
mean,25.254631,181.224208,5.530703,2.552716e+06,10471.526587,4.542736e+06,258.549657,51.575129,48.023499,53.459777,60.423928,45.067324,266.572213,57.836021,49.617839,44.313379,54.202744,60.602230,323.041424,65.423585,65.436621,64.582075,62.889794,64.709348,302.260720,59.712007,65.036621,63.213036,65.164322,49.134734,261.807633,56.932762,46.764923,52.445455,55.439194,50.225300,59.947732,140.837393,46.966123,48.043568,45.827702,77.720669,15.620926,15.492539,15.346312,15.517153,15.743739,1630.789708,361.053087,2.982504,2.457890,1.143654,68.092967,55.026672,58.884134,64.213122,50.019297,64.816895,165.574185
std,4.947334,6.914567,5.814483,5.274622e+06,21244.249045,1.013805e+07,72.295006,17.865896,19.385162,16.934545,13.959573,17.732902,76.897596,18.049328,18.180408,17.472557,14.884652,15.874732,54.933247,14.746172,14.527761,14.484351,9.006966,14.131318,49.600005,13.489431,12.017118,15.358095,12.761199,19.048341,62.827366,17.062503,20.791353,19.054488,13.695285,15.676946,11.649121,61.589358,20.179110,21.468511,21.217062,81.315093,16.860437,16.162271,15.811398,16.417675,17.162589,260.266231,40.235246,0.675781,0.778954,0.439922,11.252154,13.837064,10.161584,9.807939,16.837851,9.751233,15.821873
min,16.000000,154.940000,0.000000,0.000000e+00,0.000000,0.000000e+00,42.000000,6.000000,3.000000,5.000000,11.000000,4.000000,43.000000,5.000000,4.000000,5.000000,9.000000,5.000000,113.000000,13.000000,11.000000,14.000000,24.000000,17.000000,133.000000,12.000000,25.000000,12.000000,20.000000,4.000000,55.000000,9.000000,5.000000,2.000000,10.000000,8.000000,12.000000,20.000000,3.000000,6.000000,6.000000,12.000000,2.000000,2.000000,2.000000,2.000000,2.000000,731.000000,228.000000,1.000000,1.000000,1.000000,26.000000,17.000000,25.000000,28.000000,12.000000,27.000000,117.000000
25%,21.000000,175.260000,0.000000,3.750000e+05,1000.000000,5.037500e+05,232.000000,41.000000,33.000000,46.000000,56.000000,32.000000,232.000000,53.000000,37.000000,32.000000,45.000000,57.000000,295.000000,58.000000,59.000000,57.000000,57.000000,57.000000,271.000000,50.000000,58.000000,56.000000,58.000000,35.000000,235.000000,45.000000,26.000000,43.000000,47.000000,40.000000,53.000000,83.000000,29.000000,28.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1492.000000,333.000000,3.000000,2.000000,1.000000,62.000000,46.000000,52.000000,59.000000,35.000000,59.000000,154.000000
50%,25.000000,180.340000,4.000000,8.000000e+05,3000.000000,1.300000e+06,271.000000,56.000000,52.000000,56.000000,63.000000,47.000000,279.000000,63.000000,52.000000,43.000000,57.000000,64.000000,331.000000,68.000000,68.000000,67.000000,63.000000,67.000000,308.000000,61.000000,66.000000,66.

(11660, 63)
(9328, 78)
(2332, 78)


,age,height,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w_f,sm,ir,pac,sho,pas,dri,def,phy,weight_lbs,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,bp_lwb,bp_rb,bp_rm,bp_rw,bp_rwb,bp_st,foot_right,a_w_low,a_w_medium,d_w_low,d_w_medium
count,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000
mean,0.346805,0.517310,0.214208,0.028751,0.018752,0.028149,0.545264,0.517490,0.485939,0.548204,0.594870,0.471134,0.521696,0.575739,0.503713,0.444462,0.537510,0.608018,0.597390,0.630025,0.639125,0.616526,0.549091,0.594069,0.545612,0.576690,0.574943,0.603083,0.590415,0.498336,0.576267,0.550780,0.498485,0.536029,0.536532,0.489939,0.568546,0.496359,0.511385,0.504551,0.478946,0.158969,0.161044,0.160594,0.160507,0.152839,0.161192,0.573030,0.504090,0.497871,0.358843,0.039078,0.611564,0.501615,0.502358,0.542047,0.508603,0.577261,0.378713,0.190882,0.076653,0.005010,0.062625,0.100200,0.051603,0.049098,0.011523,0.013527,0.045591,0.082164,0.020541,0.015030,0.144790,0.747495,0.041082,0.660822,0.097194,0.735972
std,0.179891,0.133461,0.228460,0.060604,0.035891,0.064718,0.185588,0.205370,0.211838,0.196898,0.168813,0.208384,0.183690,0.201968,0.204543,0.200907,0.177829,0.179173,0.154483,0.175942,0.168139,0.174281,0.126505,0.174555,0.157343,0.158021,0.167416,0.182032,0.162246,0.213838,0.179362,0.196922,0.245049,0.208672,0.161875,0.184707,0.137383,0.250182,0.232006,0.256330,0.250724,0.199289,0.199966,0.196222,0.191712,0.188899,0.204565,0.166744,0.155322,0.169110,0.194268,0.116769,0.156632,0.181272,0.148134,0.146426,0.219072,0.145555,0.126601,0.393095,0.266107,0.070622,0.242348,0.300342,0.221280,0.216127,0.106752,0.115545,0.208649,0.274684,0.141878,0.121703,0.351977,0.434558,0.198530,0.473549,0.296297,0.440925
min,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.015190,0.000000,0.010870,0.034091,0.000000,0.000000,0.021077,0.000000,0.022222,0.022472,0.011905,0.054945,0.025641,-0.012048,0.023529,0.048780,0.098592,0.037500,0.012862,0.060241,-0.042857,-0.011765,0.064935,0.000000,0.008357,0.011494,-0.011765,0.021505,0.000000,-0.011628,0.000000,0.020325,0.022989,0.023810,0.023810,0.007026,0.011364,0.000000,-0.011628,0.000000,0.000000,0.013986,0.066148,0.000000,0.000000,0.000000,0.057971,0.039474,0.058824,0.029851,0.013158,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.185185,0.437500,0.000000,0.004167,0.001786,0.003267,0.475949,0.394886,0.315217,0.454545,0.542169,0.325581,0.439696,0.516484,0.366667,0.303371,0.428571,0.560440,0.518519,0.542169,0.564706,0.524390,0.478873,0.487500,0.446945,0.457831,0.471429,

(1996, 78)


In [19]:
### pickle test

# X, y = preprocess(validate)

# model = pickle.load(open('model.sav', 'rb'))

# preds = model.predict(X)
# print(r2_score(y, preds))